In [211]:
import json
import os
from enum import Enum
from typing import List, Optional, Literal, Dict

from IPython.display import display, Image
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_ollama import ChatOllama
from langgraph.graph import START, END, StateGraph, MessagesState
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Annotated
import operator
from src.services.retrival_engine import RetrivalEngine
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.messages import get_buffer_string

os.environ.clear()
load_dotenv()

True

In [ ]:

import pandas as pd

df_2 = pd.read_excel('urls.xlsx')
df_2

# LLM

In [ ]:
local_llm = "llama3.2:latest"
model_tested = "llama3.2:latest"
metadata = f"CRAG, {model_tested}"

# Create Index
Let's index 3 blog posts

In [ ]:
# urls = [
#     "https://lpi.oregonstate.edu/sites/lpi.oregonstate.edu/files/pdf/mic/micronutrients_for_health.pdf",
#     # "https://www.accessdata.fda.gov/scripts/InteractiveNutritionFactsLabel/assets/InteractiveNFL_Vitamins%26MineralsChart_October2021.pdf",
#
#     "https://www.hilarispublisher.com/open-access/essential-nutrients-in-human-body.pdf",
# ]
#
# # Load documents from the URLs
# docs = [PyPDFLoader(url).load() for url in urls]
# docs_list = [item for sublist in docs for item in sublist]
#
# # Initialize a text splitter with specified chunk size and overlap
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=600, chunk_overlap=100
# )
#
# # Create a retrival Engine
# retrival_engine = RetrivalEngine()
#
# # Split the documents into chunks
# doc_splits = text_splitter.split_documents(docs_list)
#
# contents = [doc.page_content for doc in doc_splits]
# metadatas = []
#
# for doc in doc_splits:
#     metadata = dict(doc.metadata)
#     metadata["content"] = "nutrition_article"
#     metadatas.append(metadata)
#
# # Add items to retrival engine in smaller batches
# batch_size = 10
# for i in range(0, len(contents), batch_size):
#     batch_contents = contents[i:i + batch_size]
#     batch_metadatas = metadatas[i:i + batch_size]
#
#     # This will use the cache for any existing embeddings
#     ids = retrival_engine.bulk_add_items(
#         contents=batch_contents,
#         metadatas=batch_metadatas,
#         item_types=["nutrition_document"] * len(batch_contents)
#     )
#
# print("All documents stored in Pinecone!")


# Building the graph

## Document retriver

In [ ]:
class UserProfile(BaseModel):
    name: Optional[str] = Field(None, description="User's full name")
    age: int = Field(..., ge=0, le=120, description="User's age")
    gender: str = Field(..., description="Male, Female, or Other")
    height_cm: int = Field(..., gt=50, lt=250, description="Height in cm")
    weight_kg: float = Field(..., gt=20, lt=300, description="Weight in kg")
    activity_level: str = Field(...,
                                description="Sedentary, Lightly active, Moderately active, Very active, Super active")
    dietary_preferences: List[Literal[
        "Vegetarian", "Vegan", "Pescatarian", "Keto", "Paleo",
        "Gluten-Free", "Dairy-Free", "Nut-Free", "Halal", "Kosher",
        "Low-Carb", "Low-Fat", "High-Protein", "Mediterranean", "FODMAP", "Sugar-Free"
    ]] = Field(default=[], description="User's dietary preferences, can be one or more.")
    allergies: List[str] = Field(default=[], description="User's allergies")
    health_conditions: List[str] = Field(default=[], description="Any medical conditions")
    weight_goal: str = Field(..., description="Lose weight, Maintain weight, Gain muscle")
    past_meals: List[str] = Field(default=[], description="Past meals")


user_profile = {
    "name": "Space Cadet",
    "age": 23,
    "gender": "Male",
    "height_cm": 183,
    "weight_kg": 65,
    "activity_level": "Lightly active",
    "dietary_preferences": ["Dairy-Free", "Low-Carb"],
    "allergies": ["Peanuts"],
    "health_conditions": ["None"],
    "weight_goal": "Maintain weight"
}

user = UserProfile(**user_profile)
past_meals = []
print(user.model_dump_json(indent=4))

In [ ]:
llm = ChatOllama(model=local_llm, format="json", temperature=0)
llm_json_mode = ChatOllama(model=local_llm, format="json", temperature=0)

# Retrieval Prompt Template for Multiple Meal Plans
retriever_prompt = PromptTemplate(
    template="""
    You are a nutritionist AI assistant that helps users generate **personalized meal recommendations** based on their profile.

    The user profile is as follows:

    - Age: {age}
    - Gender: {gender}
    - Height: {height_cm} cm
    - Weight: {weight_kg} kg
    - Activity Level: {activity_level}
    - Dietary Preferences: {dietary_preferences}
    - Allergies: {allergies}
    - Health Conditions: {health_conditions}
    - Weight Goal: {weight_goal}
    - Past Meal History (if available): {past_meals}

    ### Task:
    Generate a structured JSON response with **multiple queries** for retrieving meal plans.
    Each query should focus on **one meal category**:
    - Breakfast
    - Lunch
    - Dinner
    - Snacks

    Ensure that meals align with **dietary preferences, allergies, and weight goals** while maintaining **nutritional balance**.

    Rules:
	•	If any field is an empty array `[]`, or unspecified, substitute it with a realistic, healthy default (e.g., “Mediterranean diet”, “moderately active”, “no known allergies”, etc.).
	•	If dietary preference is missing, randomly choose a healthy eating pattern such as Mediterranean, Plant-based, Paleo, or Flexitarian.
	•	Ensure that meals align with all available preferences, allergies, and weight goals while maintaining nutritional balance.


    Your output must be a valid JSON object structured as follows:
    ```json
    {{
        "queries": [
            {{
                "meal_type": "BREAKFAST",
                "query": "Retrieve high-protein breakfast meals suitable for {gender}, {age} years old, {activity_level} activity, avoiding {allergies}."
            }},
            {{
                "meal_type": "LUNCH",
                "query": "Retrieve balanced lunch options with {dietary_preferences} for a {weight_goal} goal, avoiding {allergies}."
            }},
            {{
                "meal_type": "DINNER",
                "query": "Find nutritious dinners for {age}-year-old {gender} aiming to {weight_goal}."
            }},
            {{
                "meal_type": "SNACKS",
                "query": "Suggest healthy snack options that fit within a {dietary_preferences} diet while avoiding {allergies}."
            }}
        ]
    }}
    ```
    """,
    input_variables=[
        "age",
        "gender",
        "height_cm",
        "weight_kg",
        "activity_level",
        "dietary_preferences",
        "allergies",
        "health_conditions",
        "weight_goal",
        "past_meals"
    ],
)

# Output Parser
output_parser = JsonOutputParser()


class MealQuery(TypedDict):
    meal_type: Literal["BREAKFAST", "LUNCH", "DINNER"]
    query: str


# Function to Generate Multiple Queries
def generate_retrieval_queries(user_profile, llm) -> List[MealQuery]:
    formatted_prompt = retriever_prompt.format(**user_profile)
    response = llm.invoke(formatted_prompt)
    return output_parser.parse(response.content).get("queries", [])


# Example User Profile
# user_profile = {
#     "age": 28,
#     "gender": "Male",
#     "height_cm": 183,
#     "weight_kg": 65,
#     "activity_level": "Moderately active",
#     "dietary_preferences": [],
#     "allergies": [],
#     "health_conditions": [],
#     "weight_goal": "Maintain weight",
#     "past_meals": ["Oatmeal with fruits", "Grilled chicken with rice", "Salmon with vegetables"],
# }

user_profile = {
    "name": "Space Cadet",
    "age": 23,
    "gender": "Male",
    "height_cm": 183,
    "weight_kg": 65,
    "activity_level": "Lightly active",
    "dietary_preferences": ["Dairy-Free", "Low-Carb"],
    "allergies": ["Peanuts"],
    "health_conditions": ["None"],
    "weight_goal": "Maintain weight",
    "past_meals": ["Oatmeal with fruits", "Grilled chicken with rice", "Salmon with vegetables"],
}
# Generate queries
queries_info = generate_retrieval_queries(user_profile, llm_json_mode)
print(queries_info)

In [ ]:
# # Function to get retrival docs
# # Create a retrival Engine
# retrival_engine = RetrivalEngine()
# class RetrievedDocuments(BaseModel):
#     meal_type: str
#     query: str
#     docs: List[str]
#
#
# def retrieve_docs(queries_info, retriver):
#     all_results = []
#     for query_info in queries_info:
#         query = query_info["query"]
#         meal_type = query_info["meal_type"]
#         retreived_docs = retriver.get_retrivals(query, top_k=2)
#         all_results.append(RetrievedDocuments(meal_type=meal_type, query=query,
#                                               docs=list(map(lambda x: x["metadata"]["content"], retreived_docs))))
#
#     return all_results
#
#
# queries_info = generate_retrieval_queries(user_profile, llm_json_mode)
# results = retrieve_docs(queries_info, retrival_engine)
# results

In [ ]:
# len(results)

## Router

In [ ]:
router_instructions = """
You are an expert in routing user health profiles to the most relevant data source. Your task is to determine whether a query should be answered using a vectorstore or a web search.
	•	The vectorstore contains documents related to nutrition and food for health. Use this for queries specifically about diet, nutrition, or health-related food topics.
	•	For all other topics, especially current events, use web search as the data source.
	•	Your response must be a JSON object with a single key, datasource, whose value is either 'vectorstore' or 'websearch'. example: {"datasource": "vectorstore"} or {"datasource": "websearch"}

Ensure your decision-making is clear, accurate, and follows these rules strictly.
"""

# Testing
test_web_search = llm_json_mode.invoke(
    [SystemMessage(router_instructions), HumanMessage("What are the benefits of turmeric?")]
)
test_web_search2 = llm_json_mode.invoke(
    [SystemMessage(router_instructions), HumanMessage("Who is the president of the United States?")]
)
print(
    json.loads(test_web_search.content),
    json.loads(test_web_search2.content)
)

## Retrival Grader

## Intializing Chat Gpt

In [ ]:
from typing import Any, Union

load_dotenv()

from langchain_openai import ChatOpenAI

llm_chat = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

llm_chat_json = llm_chat.with_structured_output(method="json_mode")

In [ ]:

# doc_grader_instructions = """
# You are a grader assessing the quality and relevance of a retrieved nutrition document to a user's profile and a specified meal type.
#
# Evaluate whether the document provides useful, accurate, and practical nutritional information that aligns with health and dietary topics, as well as the specified meal type.
#
# If the document contains scientifically valid and actionable insights on nutrition and is relevant to the given meal type, grade it as relevant.
# """

doc_grader_prompt = """
You are a grader assessing the quality and relevance of a retrieved nutrition document to a user's profile and a specified meal type.

Evaluate whether the document provides useful, accurate, and practical nutritional information that aligns with health and dietary topics, as well as the specified meal type.

If the document contains scientifically valid and actionable insights on nutrition and is relevant to the given meal type, grade it as relevant.


Here is the retrieved nutrition document: \n\n {document} \n\n
Here is the user's profile: \n\n {user_profile} \n\n if the users profile
Meal type: {meal_type} \n\n
Here is the grading criteria:
- The document should be scientifically accurate and credible.
- It should be relevant to nutrition and health-related topics.
- It should provide clear, actionable, and practical guidance.
- It should align with the specified meal type ({meal_type}).

Carefully and objectively assess whether the document meets these criteria.

Your answer should include:
1. A binary score of 'yes' or 'no'.
2. A reason of why you gave that score, based on the criteria.

Return JSON with a 2 keys, binary_score and reason as to why it reached that criteria
"""


class MealTypes(Enum):
    BREAKFAST = "Breakfast"
    LUNCH = "Lunch"
    DINNER = "Dinner"
    SNACKS = "Snacks"


# class DocumentMetadata(BaseModel):
#     content: str
#     creationdate: Optional[str]
#     creator: Optional[str]
#     item_type: str
#     moddate: Optional[str]
#     page: Optional[float]
#     page_label: Optional[str]
#     producer: Optional[str]
#     source: Optional[str]
#     total_pages: Optional[float]
#     trapped: Optional[str]

#
# class RetrievedDocumentsMetadata(BaseModel):
#     id: str
#     score: float
#     metadata: DocumentMetadata


# def grade_documents(user_profile, retrieve_docs, meal_type:MealTypes) -> GradedDocuments:
#     grade_documents:List[GradedDocuments] = []
#
#     for doc in retrieve_docs:
#         doc_content = doc["metadata"]["content"]
#         formatted_prompt = doc_grader_prompt.format(document=doc_content, user_profile=user_profile, meal_type=meal_type.value)
#         graded_response = llm_chat_json.invoke(formatted_prompt)
#
#
#     return grade_documents
#
# grade_documents = grade_documents(user_profile, results["BREAKFAST"], MealTypes.BREAKFAST)
class ValidatedDocument(BaseModel):
    valid: bool
    doc: Union[Optional[Dict[str, str]], Any]
    reaseon: Optional[str]


class GradedMealTypes(BaseModel):
    meal_type: str
    query: str
    validated_docs: List[ValidatedDocument]


class GraderResponse(BaseModel):
    binary_score: Literal["yes", "no"]
    reason: str

# function to grade the documents
# def grade_documents(profile: UserProfile, retrieved_docs: List[RetrievedDocuments]) -> List[GradedMealTypes]:
#     print(f"User profile: \n {profile}")
#     result: List[GradedMealTypes] = []
#     for retrieved_documents in retrieved_docs:
#         meal_type = retrieved_documents.meal_type
#         query = retrieved_documents.query
#         to_input = GradedMealTypes(meal_type=meal_type, query=query, validated_docs=[])
#         valid_docs: List[ValidatedDocument] = []
#
#         print(f"===== checking for meal_type: {meal_type} =====")
#         for index, doc_content in enumerate(retrieved_documents.docs):
#             formatted_prompt = doc_grader_prompt.format(document=doc_content, meal_type=meal_type,
#                                                         user_profile=json.dumps(profile, indent=4))
#
#             graded_response = llm_chat_json.invoke(formatted_prompt)
#
#             print(f"\tgraded_response for document {index + 1}: {graded_response["binary_score"]}")
#             if graded_response["binary_score"] == "yes":
#                 print("\tThe document is:")
#                 print(f"\n\t\t{doc_content}")
#                 valid_docs.append(ValidatedDocument(valid=True, doc=doc_content, reaseon=graded_response["reason"]))
#             else:
#                 print(f"\t\tThe document is: \n\t\t{doc_content}")
#                 valid_docs.append(ValidatedDocument(valid=False, doc=doc_content, reaseon=graded_response["reason"]))
#
#         to_input.validated_docs = valid_docs
#
#         result.append(to_input)
#
#     return result
#
#
# grade_meal_documents = grade_documents(user_profile, results)
# grade_meal_documents


In [ ]:
len(grade_meal_documents)

In [ ]:
len(grade_meal_documents[0].validated_docs)

# Reaseach Agent to help refined retrieved documents
![Screenshot 2024-08-26 at 7.26.33 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbb164d61c93d48e604091_research-assistant1.png)

In [109]:
from langgraph.graph import add_messages
from langchain_core.messages import AnyMessage


# class Analyst(BaseModel):
#     affiliation: str = Field(
#         description="Primary affiliation of the analyst",
#     )
#     name: str = Field(
#         description="Name of the analyst",
#     )
#     role: str = Field(
#         description="Role of the analyst in the context of the topic",
#     )
#     description: str = Field(
#         description="Description of the analyst focus, concerns, and motives.",
#     )
#
#     @property
#     def person(self) -> str:
#         return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"


# class Perspectives(BaseModel):
#     analysts: List[Analyst] = Field(
#         description="Comprehensive list of analysts with their roles and affiliations.",
#     )
class Analyst(BaseModel):
    name: str
    tone: str
    theme: str
    description: str


class MealTypeAnalysts(BaseModel):
    meal_type: str
    analysts: List[Analyst]


class AnalystMessages(MessagesState):
    analyst: Analyst


class GenerateAnalystsState(TypedDict):
    user_profile: UserProfile  # The user's profile
    meal_queries: List[MealQuery]
    # topic: str  # Research topic
    max_analysts: int  # Number of analysts
    analysts: List[MealTypeAnalysts]  # Analyst asking questions
    graded_meal_documents: List[GradedMealTypes]  # List of the graded meal types


### Old prompt

meal_assistant_prompt = """
You are an AI tasked with generating a set of meal assistant personas based on user-specific dietary context. Follow these instructions carefully and respond in valid JSON format.

1. Review the user's profile to understand their dietary habits, preferences, restrictions, and goals:
{user_profile}

2. Examine the retrieved document provided below. This document has been evaluated by another AI reviewer.

Document:
{retrieved_document}

AI Review Summary:
- Query used to retrieve the document: {query_topic}
- Document validity: {document_validity}
- Feedback: {document_feedback}

3. Consider the type of meal for which these assistants are being created:
{meal_type}

4. Based on the document content, its validity, and the user profile, identify the most important themes. Themes may relate to dietary needs, health goals, preparation time, cultural relevance, food variety, or lifestyle factors.

5. Select the top {max_assistants} relevant themes for the current meal context.

6. For each selected theme, create one unique AI assistant persona. Each assistant must:
    - Have a distinct name and tone (e.g., cheerful, nurturing, analytical).
    - Focus on one specific theme informed by the query, the document, and the AI feedback.
    - Be customized to assist the user specifically for {meal_type} planning or decisions.
    - Offer suggestions or support that reflect both the user's goals and the information from the document.

Return your output as a valid JSON object matching this structure:
{{
  "analysts": [
    {{
      "name": "<assistant_name>",
      "tone": "<tone_description>",
      "theme": "<core_theme>",
      "description": "<brief description of this persona and how it helps the user for this meal>"
    }}
  ]
}}
Ensure the top-level key is "analysts".
"""

### Old analyst function
def create_analysts(state: GenerateAnalystsState):
    llm_chat_json = llm_chat.with_structured_output(method="json_mode")
    max_assistants = state["max_analysts"]
    graded_meal_documents = state["graded_meal_documents"]
    user_profile = state["user_profile"]
    result: List[MealTypeAnalysts] = []

    for graded_meal in graded_meal_documents:
        meal_type = graded_meal.meal_type
        query = graded_meal.query
        validated_docs = graded_meal.validated_docs

        for validated_doc in validated_docs:
            system_message = meal_assistant_prompt.format(
                user_profile=json.dumps(user_profile),
                retrieved_document=validated_doc.doc,
                document_validity=False,
                document_feedback=validated_doc.reaseon,
                meal_type=meal_type,
                max_assistants=max_assistants,
                query_topic=query
            )
            analysts = llm_chat_json.invoke(
                [SystemMessage(content=system_message)] + [HumanMessage(content="Generate the set of analysts.")])

            generated_analyst = MealTypeAnalysts(
                meal_type=meal_type,
                analysts=analysts["analysts"],
            )

            result.append(generated_analyst)

    # Write to state
    return {"analysts": result}

In [ ]:
queries_info

In [ ]:
meal_assistant_prompt = """
You are an AI tasked with generating a set of meal assistant personas based solely on a user's dietary context. Follow these instructions carefully and respond in valid JSON format.

1. Review the user's profile to understand their dietary habits, preferences, restrictions, and goals:
{user_profile}

2. Consider the type of meal for which these assistants are being created:
{meal_type} and the following instruction: {query}

3. Identify the most important themes from the user's profile. Themes may include dietary needs, health goals, preparation time, cultural relevance, food variety, or lifestyle considerations.

4. Select the top {max_assistants} relevant themes for the current meal context.

5. For each selected theme, create one unique AI assistant persona. Each assistant must:
    - Have a distinct name and tone (e.g., cheerful, nurturing, analytical).
    - Focus on one specific theme derived from the user profile.
    - Be customized to assist the user specifically for {meal_type} planning or decisions.
    - Offer helpful suggestions or support that reflect the user’s goals and context.

Return your output as a valid JSON object matching this structure:
[
    {{
      "name": "<assistant_name>",
      "tone": "<tone_description>",
      "theme": "<core_theme>",
      "description": "<brief description of this persona and how it helps the user for this meal>"
    }}
    ,.....
  ]
Ensure the top-level key is "analysts".
"""

llm_chat = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


def create_analysts(state: GenerateAnalystsState):
    llm_chat = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )

    llm_chat_json = llm_chat.with_structured_output(method="json_mode")
    max_assistants = state["max_analysts"]
    meal_queries = state["meal_queries"]
    user_profile = state["user_profile"]

    result: List[MealTypeAnalysts] = []

    for meal in meal_queries:
        meal_type = meal["meal_type"]
        query = meal["query"]

        system_message = meal_assistant_prompt.format(
            user_profile=json.dumps(user_profile),
            meal_type=meal_type,
            query=query,
            max_assistants=max_assistants,
        )

        analysts = llm_chat_json.invoke(
            [SystemMessage(content=system_message)] + [HumanMessage(content="Generate the set of analysts.")])

        generated_analyst = MealTypeAnalysts(
            meal_type=meal_type,
            analysts=analysts["analysts"],
        )

        result.append(generated_analyst)

    # Write to state
    return {"analysts": result}


builder = StateGraph(GenerateAnalystsState)
builder.add_node("create_analyst", create_analysts)
builder.add_edge(START, "create_analyst")
builder.add_edge("create_analyst", END)

create_analyst_graph = builder.compile()

In [ ]:
display(Image(create_analyst_graph.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
queries_info

In [ ]:
# # Input
# max_analysts = 3
# # topic = grade_meal_documents[0].query
# # graded_meal_documents = grade_meal_documents
# # thread = {"configurable": {"thread_id": "1"}}
#
# # Run the graph until the first interruption
# for event in create_analyst_graph.stream({"max_analysts": max_analysts,
#                                           "meal_queries": queries_info, "user_profile": user_profile},
#                                          stream_mode="values"):
#     # Review
#     meal_type_analysts_event: List[MealTypeAnalysts] = event
#
#     meal_type_analysts: MealTypeAnalysts = meal_type_analysts_event.get("analysts", "")
#
#     if meal_type_analysts:
#         for analysts in meal_type_analysts:
#             meal_type_analysts = analysts.analysts
#             for analyst_meta in meal_type_analysts:
#                 print(f"Name: {analyst_meta['name']}")
#                 print(f"Tone: {analyst_meta['tone']}")
#                 print(f"Theme: {analyst_meta['theme']}")
#                 print(f"Description: {analyst_meta['description']}")
#                 print("-" * 50)

In [125]:
# meal_type_analysts = meal_type_analysts_event.get("analysts")
meal_type_analysts = [MealTypeAnalysts(meal_type='BREAKFAST', analysts=[
    Analyst(name='Protein Pete', tone='Energetic', theme='High-Protein Diet',
            description='Protein Pete is here to pump up your breakfast with high-protein options that fit your dairy-free and low-carb lifestyle. He suggests meals like scrambled eggs with spinach and turkey bacon, or a tofu scramble with avocado, ensuring you start your day with the energy you need.'),
    Analyst(name='Carb-Conscious Carla', tone='Practical', theme='Low-Carb Options',
            description='Carb-Conscious Carla focuses on keeping your breakfast low in carbs while still being satisfying and delicious. She recommends dishes like a vegetable omelette with a side of smoked salmon or chia seed pudding with almond milk and berries, helping you maintain your dietary preferences.'),
    Analyst(name='Allergy-Aware Alex', tone='Caring', theme='Allergy Management',
            description='Allergy-Aware Alex ensures your breakfast is safe and peanut-free. He provides guidance on ingredient substitutions and meal ideas like coconut yogurt with nuts and seeds or a smoothie bowl with almond butter, so you can enjoy your meal worry-free.')]),
                      MealTypeAnalysts(meal_type='LUNCH', analysts=[
                          Analyst(name='Carb-Conscious Carl', tone='Analytical', theme='Low-Carb Diet',
                                  description='Carl is here to help you maintain your low-carb lifestyle while ensuring your lunch is both satisfying and nutritious. He suggests meals like grilled chicken salad with a variety of greens and a vinaigrette dressing, or a turkey lettuce wrap with avocado and tomato, ensuring you stay on track with your dietary preferences.'),
                          Analyst(name='Dairy-Free Daisy', tone='Cheerful', theme='Dairy-Free Options',
                                  description='Daisy is your go-to for delicious dairy-free lunch ideas. She loves to recommend meals like a quinoa and black bean bowl with roasted vegetables, or a zesty lemon herb shrimp with a side of sautéed spinach, making sure your meals are both tasty and free from dairy.'),
                          Analyst(name='Balanced Ben', tone='Nurturing', theme='Weight Maintenance',
                                  description='Ben focuses on helping you maintain your weight with balanced lunch options. He suggests meals like a grilled salmon with a side of asparagus and a small portion of sweet potato, or a chicken stir-fry with bell peppers and broccoli, ensuring you get the right nutrients without overindulging.')]),
                      MealTypeAnalysts(meal_type='DINNER', analysts=[
                          Analyst(name='Carb-Conscious Carl', tone='Analytical', theme='Low-Carb Diet',
                                  description='Carl is here to help you maintain your low-carb lifestyle while ensuring your dinners are both satisfying and nutritious. He suggests meals like zucchini noodles with turkey meatballs or a hearty chicken Caesar salad without croutons, keeping your carb intake in check.'),
                          Analyst(name='Dairy-Free Delilah', tone='Nurturing', theme='Dairy-Free Options',
                                  description='Delilah understands your need to avoid dairy and is ready to offer comforting, dairy-free dinner ideas. She recommends dishes like coconut curry shrimp or a creamy avocado pasta, ensuring you enjoy delicious meals without dairy.'),
                          Analyst(name='Balanced Benny', tone='Cheerful', theme='Weight Maintenance',
                                  description='Benny is all about balance and helping you maintain your weight with delightful dinners. He suggests meals like grilled salmon with a side of roasted vegetables or a quinoa and black bean bowl, ensuring you get the right nutrients to keep your weight stable.')]),
                      MealTypeAnalysts(meal_type='SNACKS', analysts=[
                          Analyst(name='Carb-Conscious Carl', tone='Analytical', theme='Low-Carb Diet',
                                  description='Carl is here to help you navigate the world of low-carb snacks. He provides detailed insights into carb content and suggests options like almond butter celery sticks or turkey roll-ups to keep your snack choices aligned with your dietary preferences.'),
                          Analyst(name='Dairy-Free Daisy', tone='Cheerful', theme='Dairy-Free Options',
                                  description='Daisy is your go-to for all things dairy-free. With a sunny disposition, she offers creative snack ideas like hummus with veggie sticks or coconut yogurt with berries, ensuring you enjoy delicious snacks without any dairy.'),
                          Analyst(name='Allergy-Aware Alex', tone='Nurturing', theme='Allergy Considerations',
                                  description='Alex is dedicated to keeping your snacks safe and peanut-free. With a caring approach, he suggests options like apple slices with sunflower seed butter or roasted chickpeas, ensuring your snacks are both safe and satisfying.')])]
meal_type_analysts

[MealTypeAnalysts(meal_type='BREAKFAST', analysts=[Analyst(name='Protein Pete', tone='Energetic', theme='High-Protein Diet', description='Protein Pete is here to pump up your breakfast with high-protein options that fit your dairy-free and low-carb lifestyle. He suggests meals like scrambled eggs with spinach and turkey bacon, or a tofu scramble with avocado, ensuring you start your day with the energy you need.'), Analyst(name='Carb-Conscious Carla', tone='Practical', theme='Low-Carb Options', description='Carb-Conscious Carla focuses on keeping your breakfast low in carbs while still being satisfying and delicious. She recommends dishes like a vegetable omelette with a side of smoked salmon or chia seed pudding with almond milk and berries, helping you maintain your dietary preferences.'), Analyst(name='Allergy-Aware Alex', tone='Caring', theme='Allergy Management', description='Allergy-Aware Alex ensures your breakfast is safe and peanut-free. He provides guidance on ingredient subs

# Conduct Interview

In [262]:
from langgraph.graph import add_messages


class SourcedDocuments(TypedDict):
    meal_type_analysts: MealTypeAnalysts
    context: Annotated[list, operator.add]


class RecommendedMeal(TypedDict):
    meal_name: str
    meal_type: str
    ingredients: list[str]
    portion: str
    goal_support: str


class RecommendedMeals(TypedDict):
    user_profile: UserProfile
    recommended_meals: Annotated[list[RecommendedMeal], add_messages]


class InterviewState(BaseModel):
    user_profile: UserProfile
    max_num_turns: int
    web_documents: Annotated[list[SourcedDocuments], operator.add] = Field(default_factory=list)
    wiki_documents: Annotated[list[SourcedDocuments], operator.add] = Field(default_factory=list)
    sourced_documents: Annotated[list[SourcedDocuments], operator.add] = Field(default_factory=list)
    meal_type_analysts: List[MealTypeAnalysts] = Field(default_factory=list)
    interview: str = ""
    analysts_messages: List[AnalystMessages] = Field(default_factory=list)
    recommended_meals: RecommendedMeals = Field(default_factory=dict)


class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Search query for retrieval.")

In [ ]:
weight_maintenance_max = Analyst(
    name="Weight-Maintenance Max",
    tone="supportive",
    theme="Weight Maintenance",
    description=(
        "Max is focused on helping you maintain your current weight with balanced lunch options. "
        "He suggests meals like a grilled salmon fillet with a side of steamed broccoli and a small portion of sweet potato, "
        "or a tofu stir-fry with mixed vegetables. Max ensures your meals are portioned correctly to support your weight maintenance goal."
    )
)
weight_maintenance_max["theme"]

In [141]:
question_template = """You are an analyst tasked with interviewing an expert to learn about a specific topic.

Your goal is boil down to interesting and specific insights related to your topic.

1. Interesting: Insights that people will find surprising or non-obvious.

2. Specific: Insights that avoid generalities and include specific examples from the expert.

Here is your topic of focus and set of goals: {goals}

Begin by introducing yourself using a name that fits your persona, and then ask your question.

Continue to ask questions to drill down and refine your understanding of the topic.

When you are satisfied with your understanding, complete the interview with: "Thank you so much for your help!"

Remember to stay in character throughout your response, reflecting the persona and goals provided to you.
"""


def generate_question(state: InterviewState):
    """ Node to generate a question """
    llm_chat = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )

    meal_type_analysts: List[MealTypeAnalysts] = state.meal_type_analysts
    analysts_messages = state.analysts_messages

    for meal_type_analyst in meal_type_analysts:
        analysts = meal_type_analyst.analysts
        for analyst in analysts:
            goals = f"Explore expert strategies related to {analyst.theme.lower()} — specifically, {analyst.description}"
            system_message = question_template.replace("{goals}", goals)
            question = llm_chat.invoke([
                SystemMessage(content=system_message),
                HumanMessage(content="Let's begin.")
            ])

            analysts_messages.append(
                AnalystMessages(
                    analyst=analyst,
                    messages=[question],
                )
            )

    # Write messages to state
    return {"analysts_messages": [analysts_messages]}

In [ ]:
# # Testing
# analysts_messages = []
#
# for meal_type_analyst in meal_type_analysts:
#     analysts = meal_type_analyst.analysts
#     for analyst_meta in analysts:
#         goals = f"Explore expert strategies related to {analyst_meta.theme.lower()} — specifically, {analyst_meta.description}"
#         system_message = question_template.replace("{goals}", goals)
#         question = llm_chat.invoke([
#             SystemMessage(content=system_message),
#             HumanMessage(content="Let's begin.")
#         ])
#
#         analysts_messages.append(
#             AnalystMessages(
#                 analyst=analyst_meta,
#                 messages=[question],
#             )
#         )

In [112]:
analysts_messages = [{'analyst': Analyst(name='Protein Pete', tone='Energetic', theme='High-Protein Diet',
                                         description='Protein Pete is here to pump up your breakfast with high-protein options that fit your dairy-free and low-carb lifestyle. He suggests meals like scrambled eggs with spinach and turkey bacon, or a tofu scramble with avocado, ensuring you start your day with the energy you need.'),
                      'messages': [AIMessage(
                          content="Hello, I'm Nutrition Nick, and I'm here to dive deep into the world of high-protein breakfasts that are both dairy-free and low-carb. Protein Pete, I'm excited to learn from your expertise! To start, could you share why it's important to focus on high-protein breakfasts, especially for those following a dairy-free and low-carb lifestyle?",
                          additional_kwargs={'refusal': None}, response_metadata={
                              'token_usage': {'completion_tokens': 70, 'prompt_tokens': 227, 'total_tokens': 297,
                                              'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                            'audio_tokens': 0, 'reasoning_tokens': 0,
                                                                            'rejected_prediction_tokens': 0},
                                              'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
                              'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6889ffe71',
                              'finish_reason': 'stop', 'logprobs': None},
                          id='run-885a4586-43e2-4939-836f-bb4297f1ef07-0',
                          usage_metadata={'input_tokens': 227, 'output_tokens': 70, 'total_tokens': 297,
                                          'input_token_details': {'audio': 0, 'cache_read': 0},
                                          'output_token_details': {'audio': 0, 'reasoning': 0}})]},
                     {'analyst': Analyst(name='Carb-Conscious Carla', tone='Practical', theme='Low-Carb Options',
                                         description='Carb-Conscious Carla focuses on keeping your breakfast low in carbs while still being satisfying and delicious. She recommends dishes like a vegetable omelette with a side of smoked salmon or chia seed pudding with almond milk and berries, helping you maintain your dietary preferences.'),
                      'messages': [AIMessage(
                          content="Hello, I'm Nutritious Nate, and I'm eager to learn more about low-carb breakfast options that are both satisfying and delicious. Carla, could you share some specific strategies or tips you use to ensure your breakfast dishes are not only low in carbs but also flavorful and filling?",
                          additional_kwargs={'refusal': None}, response_metadata={
                              'token_usage': {'completion_tokens': 56, 'prompt_tokens': 222, 'total_tokens': 278,
                                              'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                            'audio_tokens': 0, 'reasoning_tokens': 0,
                                                                            'rejected_prediction_tokens': 0},
                                              'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
                              'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6889ffe71',
                              'finish_reason': 'stop', 'logprobs': None},
                          id='run-a8cb6f9f-3686-4339-b682-6b1475da4329-0',
                          usage_metadata={'input_tokens': 222, 'output_tokens': 56, 'total_tokens': 278,
                                          'input_token_details': {'audio': 0, 'cache_read': 0},
                                          'output_token_details': {'audio': 0, 'reasoning': 0}})]},
                     {'analyst': Analyst(name='Allergy-Aware Alex', tone='Caring', theme='Allergy Management',
                                         description='Allergy-Aware Alex ensures your breakfast is safe and peanut-free. He provides guidance on ingredient substitutions and meal ideas like coconut yogurt with nuts and seeds or a smoothie bowl with almond butter, so you can enjoy your meal worry-free.'),
                      'messages': [AIMessage(
                          content="Hello, I'm Curious Casey, and I'm eager to learn more about allergy management, specifically when it comes to ensuring a safe and peanut-free breakfast. Allergy-Aware Alex, could you share some expert strategies for identifying hidden peanut ingredients in common breakfast foods?",
                          additional_kwargs={'refusal': None}, response_metadata={
                              'token_usage': {'completion_tokens': 51, 'prompt_tokens': 217, 'total_tokens': 268,
                                              'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                            'audio_tokens': 0, 'reasoning_tokens': 0,
                                                                            'rejected_prediction_tokens': 0},
                                              'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
                              'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6889ffe71',
                              'finish_reason': 'stop', 'logprobs': None},
                          id='run-e4225b8f-130a-4be7-b71b-e1cd237f15eb-0',
                          usage_metadata={'input_tokens': 217, 'output_tokens': 51, 'total_tokens': 268,
                                          'input_token_details': {'audio': 0, 'cache_read': 0},
                                          'output_token_details': {'audio': 0, 'reasoning': 0}})]},
                     {'analyst': Analyst(name='Carb-Conscious Carl', tone='Analytical', theme='Low-Carb Diet',
                                         description='Carl is here to help you maintain your low-carb lifestyle while ensuring your lunch is both satisfying and nutritious. He suggests meals like grilled chicken salad with a variety of greens and a vinaigrette dressing, or a turkey lettuce wrap with avocado and tomato, ensuring you stay on track with your dietary preferences.'),
                      'messages': [AIMessage(
                          content="Hello, I'm Alex, a health and nutrition enthusiast eager to learn more about maintaining a low-carb lifestyle, especially when it comes to lunch options. Carl, could you share some specific strategies or tips for preparing a low-carb lunch that is both satisfying and nutritious?",
                          additional_kwargs={'refusal': None}, response_metadata={
                              'token_usage': {'completion_tokens': 53, 'prompt_tokens': 232, 'total_tokens': 285,
                                              'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                            'audio_tokens': 0, 'reasoning_tokens': 0,
                                                                            'rejected_prediction_tokens': 0},
                                              'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
                              'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6889ffe71',
                              'finish_reason': 'stop', 'logprobs': None},
                          id='run-518110f5-eac5-4aaf-9e62-aa846434e60d-0',
                          usage_metadata={'input_tokens': 232, 'output_tokens': 53, 'total_tokens': 285,
                                          'input_token_details': {'audio': 0, 'cache_read': 0},
                                          'output_token_details': {'audio': 0, 'reasoning': 0}})]},
                     {'analyst': Analyst(name='Dairy-Free Daisy', tone='Cheerful', theme='Dairy-Free Options',
                                         description='Daisy is your go-to for delicious dairy-free lunch ideas. She loves to recommend meals like a quinoa and black bean bowl with roasted vegetables, or a zesty lemon herb shrimp with a side of sautéed spinach, making sure your meals are both tasty and free from dairy.'),
                      'messages': [AIMessage(
                          content="Hello, Daisy! My name is Alex, and I'm thrilled to dive into the world of delicious dairy-free lunch ideas with you. I've heard you have some fantastic strategies for creating meals that are both tasty and free from dairy. Could you start by sharing what inspired you to focus on dairy-free options, and how you ensure these meals are flavorful and satisfying?",
                          additional_kwargs={'refusal': None}, response_metadata={
                              'token_usage': {'completion_tokens': 72, 'prompt_tokens': 227, 'total_tokens': 299,
                                              'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                            'audio_tokens': 0, 'reasoning_tokens': 0,
                                                                            'rejected_prediction_tokens': 0},
                                              'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
                              'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6889ffe71',
                              'finish_reason': 'stop', 'logprobs': None},
                          id='run-99390616-1c12-4f73-adda-6310efecc086-0',
                          usage_metadata={'input_tokens': 227, 'output_tokens': 72, 'total_tokens': 299,
                                          'input_token_details': {'audio': 0, 'cache_read': 0},
                                          'output_token_details': {'audio': 0, 'reasoning': 0}})]},
                     {'analyst': Analyst(name='Balanced Ben', tone='Nurturing', theme='Weight Maintenance',
                                         description='Ben focuses on helping you maintain your weight with balanced lunch options. He suggests meals like a grilled salmon with a side of asparagus and a small portion of sweet potato, or a chicken stir-fry with bell peppers and broccoli, ensuring you get the right nutrients without overindulging.'),
                      'messages': [AIMessage(
                          content="Hello, my name is Alex, and I'm an analyst interested in learning more about effective strategies for weight maintenance, particularly through balanced lunch options. I understand you have some expertise in this area, Ben. Could you share why you recommend meals like grilled salmon with asparagus and sweet potato, or chicken stir-fry with bell peppers and broccoli? What makes these meals particularly effective for maintaining weight?",
                          additional_kwargs={'refusal': None}, response_metadata={
                              'token_usage': {'completion_tokens': 78, 'prompt_tokens': 228, 'total_tokens': 306,
                                              'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                            'audio_tokens': 0, 'reasoning_tokens': 0,
                                                                            'rejected_prediction_tokens': 0},
                                              'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
                              'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6889ffe71',
                              'finish_reason': 'stop', 'logprobs': None},
                          id='run-73ce4f00-2227-4a20-a4c8-91fea70deeeb-0',
                          usage_metadata={'input_tokens': 228, 'output_tokens': 78, 'total_tokens': 306,
                                          'input_token_details': {'audio': 0, 'cache_read': 0},
                                          'output_token_details': {'audio': 0, 'reasoning': 0}})]},
                     {'analyst': Analyst(name='Carb-Conscious Carl', tone='Analytical', theme='Low-Carb Diet',
                                         description='Carl is here to help you maintain your low-carb lifestyle while ensuring your dinners are both satisfying and nutritious. He suggests meals like zucchini noodles with turkey meatballs or a hearty chicken Caesar salad without croutons, keeping your carb intake in check.'),
                      'messages': [AIMessage(
                          content="Hello, I'm Alex, a health and nutrition enthusiast eager to learn more about maintaining a low-carb lifestyle. Carl, could you share some specific strategies or tips for creating satisfying and nutritious low-carb dinners that go beyond the usual options? For instance, how can we make zucchini noodles with turkey meatballs more exciting or a chicken Caesar salad more filling without adding carbs?",
                          additional_kwargs={'refusal': None}, response_metadata={
                              'token_usage': {'completion_tokens': 73, 'prompt_tokens': 221, 'total_tokens': 294,
                                              'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                            'audio_tokens': 0, 'reasoning_tokens': 0,
                                                                            'rejected_prediction_tokens': 0},
                                              'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
                              'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6889ffe71',
                              'finish_reason': 'stop', 'logprobs': None},
                          id='run-01043e37-8818-4aa5-a353-b83171a4d13d-0',
                          usage_metadata={'input_tokens': 221, 'output_tokens': 73, 'total_tokens': 294,
                                          'input_token_details': {'audio': 0, 'cache_read': 0},
                                          'output_token_details': {'audio': 0, 'reasoning': 0}})]},
                     {'analyst': Analyst(name='Dairy-Free Delilah', tone='Nurturing', theme='Dairy-Free Options',
                                         description='Delilah understands your need to avoid dairy and is ready to offer comforting, dairy-free dinner ideas. She recommends dishes like coconut curry shrimp or a creamy avocado pasta, ensuring you enjoy delicious meals without dairy.'),
                      'messages': [AIMessage(
                          content="Hello, Delilah! My name is Alex, and I'm thrilled to learn from your expertise in creating comforting, dairy-free dinner options. I'm particularly interested in understanding how you approach crafting these meals to ensure they are both delicious and satisfying. Could you share some specific strategies or ingredients you rely on to replace dairy in dishes like coconut curry shrimp or creamy avocado pasta?",
                          additional_kwargs={'refusal': None}, response_metadata={
                              'token_usage': {'completion_tokens': 73, 'prompt_tokens': 213, 'total_tokens': 286,
                                              'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                            'audio_tokens': 0, 'reasoning_tokens': 0,
                                                                            'rejected_prediction_tokens': 0},
                                              'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
                              'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6889ffe71',
                              'finish_reason': 'stop', 'logprobs': None},
                          id='run-9adcfc1c-7549-4ecc-86c9-76c3965292ec-0',
                          usage_metadata={'input_tokens': 213, 'output_tokens': 73, 'total_tokens': 286,
                                          'input_token_details': {'audio': 0, 'cache_read': 0},
                                          'output_token_details': {'audio': 0, 'reasoning': 0}})]},
                     {'analyst': Analyst(name='Balanced Benny', tone='Cheerful', theme='Weight Maintenance',
                                         description='Benny is all about balance and helping you maintain your weight with delightful dinners. He suggests meals like grilled salmon with a side of roasted vegetables or a quinoa and black bean bowl, ensuring you get the right nutrients to keep your weight stable.'),
                      'messages': [AIMessage(
                          content="Hello, I'm Alex, a health and wellness analyst. I'm eager to learn more about your approach to weight maintenance through balanced and delightful dinners. Could you share some specific strategies or tips you use to ensure meals like grilled salmon with roasted vegetables or a quinoa and black bean bowl are both nutritious and satisfying?",
                          additional_kwargs={'refusal': None}, response_metadata={
                              'token_usage': {'completion_tokens': 61, 'prompt_tokens': 218, 'total_tokens': 279,
                                              'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                            'audio_tokens': 0, 'reasoning_tokens': 0,
                                                                            'rejected_prediction_tokens': 0},
                                              'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
                              'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6889ffe71',
                              'finish_reason': 'stop', 'logprobs': None},
                          id='run-679a6299-b304-4cc1-9b0a-d1f5a748d5ae-0',
                          usage_metadata={'input_tokens': 218, 'output_tokens': 61, 'total_tokens': 279,
                                          'input_token_details': {'audio': 0, 'cache_read': 0},
                                          'output_token_details': {'audio': 0, 'reasoning': 0}})]},
                     {'analyst': Analyst(name='Carb-Conscious Carl', tone='Analytical', theme='Low-Carb Diet',
                                         description='Carl is here to help you navigate the world of low-carb snacks. He provides detailed insights into carb content and suggests options like almond butter celery sticks or turkey roll-ups to keep your snack choices aligned with your dietary preferences.'),
                      'messages': [AIMessage(
                          content="Hello, Carl. My name is Alex, and I'm eager to learn more about expert strategies for low-carb snacks. I've heard that almond butter celery sticks and turkey roll-ups are great options, but I'm curious about the specifics. Could you tell me more about the carb content in these snacks and why they are particularly good choices for someone on a low-carb diet?",
                          additional_kwargs={'refusal': None}, response_metadata={
                              'token_usage': {'completion_tokens': 73, 'prompt_tokens': 216, 'total_tokens': 289,
                                              'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                            'audio_tokens': 0, 'reasoning_tokens': 0,
                                                                            'rejected_prediction_tokens': 0},
                                              'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
                              'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6889ffe71',
                              'finish_reason': 'stop', 'logprobs': None},
                          id='run-46c1d5b2-5897-46c5-92f4-fcf8b02fb045-0',
                          usage_metadata={'input_tokens': 216, 'output_tokens': 73, 'total_tokens': 289,
                                          'input_token_details': {'audio': 0, 'cache_read': 0},
                                          'output_token_details': {'audio': 0, 'reasoning': 0}})]},
                     {'analyst': Analyst(name='Dairy-Free Daisy', tone='Cheerful', theme='Dairy-Free Options',
                                         description='Daisy is your go-to for all things dairy-free. With a sunny disposition, she offers creative snack ideas like hummus with veggie sticks or coconut yogurt with berries, ensuring you enjoy delicious snacks without any dairy.'),
                      'messages': [AIMessage(
                          content="Hello Daisy, I'm Alex, and I'm thrilled to dive into the world of dairy-free options with you. I've heard you have some fantastic snack ideas that are both delicious and dairy-free. Could you share some of your favorite creative snack ideas and what makes them stand out?",
                          additional_kwargs={'refusal': None}, response_metadata={
                              'token_usage': {'completion_tokens': 55, 'prompt_tokens': 214, 'total_tokens': 269,
                                              'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                            'audio_tokens': 0, 'reasoning_tokens': 0,
                                                                            'rejected_prediction_tokens': 0},
                                              'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
                              'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6889ffe71',
                              'finish_reason': 'stop', 'logprobs': None},
                          id='run-b85a22d1-a4e8-4193-b1e0-6835f80800bb-0',
                          usage_metadata={'input_tokens': 214, 'output_tokens': 55, 'total_tokens': 269,
                                          'input_token_details': {'audio': 0, 'cache_read': 0},
                                          'output_token_details': {'audio': 0, 'reasoning': 0}})]},
                     {'analyst': Analyst(name='Allergy-Aware Alex', tone='Nurturing', theme='Allergy Considerations',
                                         description='Alex is dedicated to keeping your snacks safe and peanut-free. With a caring approach, he suggests options like apple slices with sunflower seed butter or roasted chickpeas, ensuring your snacks are both safe and satisfying.'),
                      'messages': [AIMessage(
                          content="Hello, I'm Jamie, an analyst keen on understanding expert strategies for allergy considerations, particularly in creating safe, peanut-free snacks. Alex, could you share why apple slices with sunflower seed butter and roasted chickpeas are excellent choices for those with peanut allergies?",
                          additional_kwargs={'refusal': None}, response_metadata={
                              'token_usage': {'completion_tokens': 51, 'prompt_tokens': 212, 'total_tokens': 263,
                                              'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                            'audio_tokens': 0, 'reasoning_tokens': 0,
                                                                            'rejected_prediction_tokens': 0},
                                              'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
                              'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a6889ffe71',
                              'finish_reason': 'stop', 'logprobs': None},
                          id='run-5ecf2f51-c60c-4389-8f18-2ed3e778a8f0-0',
                          usage_metadata={'input_tokens': 212, 'output_tokens': 51, 'total_tokens': 263,
                                          'input_token_details': {'audio': 0, 'cache_read': 0},
                                          'output_token_details': {'audio': 0, 'reasoning': 0}})]}]
analysts_messages

[{'analyst': Analyst(name='Protein Pete', tone='Energetic', theme='High-Protein Diet', description='Protein Pete is here to pump up your breakfast with high-protein options that fit your dairy-free and low-carb lifestyle. He suggests meals like scrambled eggs with spinach and turkey bacon, or a tofu scramble with avocado, ensuring you start your day with the energy you need.'),
  'messages': [AIMessage(content="Hello, I'm Nutrition Nick, and I'm here to dive deep into the world of high-protein breakfasts that are both dairy-free and low-carb. Protein Pete, I'm excited to learn from your expertise! To start, could you share why it's important to focus on high-protein breakfasts, especially for those following a dairy-free and low-carb lifestyle?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 227, 'total_tokens': 297, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0

In [ ]:
AnalystMessages

# Generate Answers: Parallelization

Now, we create nodes to search the web and wikipedia.

We'll also create a node to answer analyst questions.

Finally, we'll create nodes to save the full interview and to write a summary ("section") of the interview.

In [264]:
import logging

logging.basicConfig(level=logging.INFO)

tavily_search = TavilySearchResults(max_results=3)

# Search query writing
search_instructions = SystemMessage(content=f"""You will be given a conversation between an analyst and an expert.
ii
Your goal is to generate a well-structured query for use in retrieval and / or web-search related to the conversation.

First, analyze the full conversation.

Pay particular attention to the final question posed by the analyst.

Convert this final question into a well-structured web search query""")


def search_web(state: InterviewState):
    """ Retrieve docs from web search """
    llm_chat = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )

    structured_llm = llm_chat.with_structured_output(SearchQuery)
    analysts_messages: List[AnalystMessages] = state.analysts_messages

    logging.info(f"Here mosima (analysts_messages): {analysts_messages})")

    all_sourced_documents = []

    for analyst in analysts_messages[0]:
        logging.info("Here Mosima", analyst)
        messages = analyst.get("messages")
        search_query = structured_llm.invoke([search_instructions] + messages)
        # Search
        search_docs = tavily_search.invoke(search_query.search_query)
        # Format
        formatted_search_docs = "\n\n---\n\n".join(
            [
                f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
                for doc in search_docs
            ]
        )

        all_sourced_documents.append(SourcedDocuments(
            meal_type_analysts=analyst,
            context=[formatted_search_docs],
        ))

    return {"web_documents": all_sourced_documents}


def search_wikipedia(state: InterviewState):
    """ Retrieve docs from wikipedia """

    llm_chat = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )

    structured_llm = llm_chat.with_structured_output(SearchQuery)
    analysts_messages: List[AnalystMessages] = state.analysts_messages
    # sourced_documents: List[SourcedDocuments] = state.sourced_documents

    all_sourced_documents = []


    for analyst in analysts_messages[0]:
        messages = analyst["messages"]
        search_query = structured_llm.invoke([search_instructions] + messages)
        # Search
        search_docs = WikipediaLoader(query=search_query.search_query,
                                      load_max_docs=2).load()

        # Format
        formatted_search_docs = "\n\n---\n\n".join(
            [
                f'<Document href="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
                for doc in search_docs
            ]
        )

    return {"wiki_documents": all_sourced_documents}

def combine_documents(state: InterviewState):
    """Combine documents from different sources"""
    combined_docs = state.web_documents + state.wiki_documents
    return {"sourced_documents": combined_docs}

answer_instructions = """You are an expert being interviewed by an analyst.

Here is analyst area of focus: {goals}.

You goal is to answer a question posed by the interviewer.

To answer question, use this context:

{context}

When answering questions, follow these guidelines:

1. Use only the information provided in the context.

2. Do not introduce external information or make assumptions beyond what is explicitly stated in the context.

3. The context contain sources at the topic of each individual document.

4. Include these sources your answer next to any relevant statements. For example, for source # 1 use [1].

5. List your sources in order at the bottom of your answer. [1] Source 1, [2] Source 2, etc

6. If the source is: <Document source="assistant/docs/llama3_1.pdf" page="7"/>' then just list:

[1] assistant/docs/llama3_1.pdf, page 7

And skip the addition of the brackets as well as the Document source preamble in your citation."""


def generate_answer(state: InterviewState):
    """ Node to answer a question """
    llm_chat = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )

    # Get state
    sourced_documents: List[SourcedDocuments] = state.sourced_documents
    analysts_messages: List[AnalystMessages] = state.analysts_messages

    for analyst in analysts_messages:
        messages = analyst["messages"]
        analyst_name = analyst["name"]

        i = next(
            (i for i, analyst_message in enumerate(analysts_messages) if
             analyst_message["analyst"]["name"] == analyst_message["name"]),
            -1
        )
        context = list(
            filter(lambda doc: doc["analyst"]["name"] == analyst_name, sourced_documents)
        )

        # Answer question
        system_message = answer_instructions.format(goals=analyst.persona, context=context)
        answer = llm_chat.invoke([SystemMessage(content=system_message)] + messages)

        # Name the message as coming from the expert
        answer.name = "expert"
        # Answer the question in the user stte
        analysts_messages[i]["messages"] = [answer]

    return {"analysts_messages": analysts_messages}


def route_messages(state: InterviewState,
                   name: str = "expert"):
    """ Route between question and answer """
    analysts_messages: List[AnalystMessages] = state.analysts_messages
    max_num_turns = state.max_num_turns

    for analyst_message in analysts_messages:
        messages = analyst_message

        # Check the number of expert answers
        num_responses = len(
            [m for m in messages if isinstance(m, AIMessage) and m.name == name]
        )

        if num_responses < max_num_turns:
            return "ask_question"

        # This router is run after each question - answer pair
        # Get the last question asked to check if it signals the end of discussion
        last_question = messages[-2]

        if "Thank you so much for your help" in last_question.content:
            return "ask_question"

    return "save_interviews"


def save_interview(state: InterviewState):
    """ Save interviews """

    # Get messages
    # messages = state["messages"]
    #
    # # Convert interview to a string
    # interview = get_buffer_string(messages)

    place_holder = "**** Everything is good, will implement save interview soon ****"

    # Save to interviews key
    return {"interview": place_holder}


json_writer_instructions = """You are a helpful assistant that extracts food recommendations from source documents.

You will be given a set of documents inside <Document> tags. Your task is to return a list of JSON objects representing food recommendations. Follow these guidelines:

1. Extract all specific food or meal suggestions from the content.
2. For each recommendation, create a JSON object with the following fields:
    - "meal_name": A short title for the meal.
    - "meal_type": One of the following values (must be in ALL CAPS exactly as written): "BREAKFAST", "LUNCH", "DINNER", or "SNACKS".
    - "ingredients": A list of main ingredients.
    - "portion": A short description of the portion size.
    - "goal_support": A brief explanation of how this meal supports weight maintenance.

3. Use only the information found in the provided documents. Do not make up any meals, ingredients, or claims.
4. Return only a valid JSON array — no explanation, no markdown, no preamble, just JSON.

Example format:

[
  {
    "meal_name": "Avocado Toast",
    "meal_type": "BREAKFAST",
    "ingredients": ["whole grain bread", "avocado", "egg"],
    "portion": "1 slice toast, 1/2 avocado, 1 egg",
    "goal_support": "Provides healthy fats and protein to support weight maintenance"
  },
  ...
]
"""


def write_recommendations(state: InterviewState):
    """ Extract food recommendations as JSON """
    llm_chat = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
    llm_chat_json = llm_chat.with_structured_output(method="json_mode")

    # sourced_documents: List[SourcedDocuments] = state.sourced_documents

    analysts_messages: List[AnalystMessages] = state.analysts_messages

    recommended_meals = state.recommended_meals

    recommended_meals["user_profile"] = state.user_profile

    for analyst_message in analysts_messages:
        analyst = analyst_message.get("analyst")
        messages = analyst_message["messages"]

        system_message = json_writer_instructions.format(focus=analyst.description)

        json_output: RecommendedMeal = llm_chat_json.invoke([
            SystemMessage(content=system_message),
            HumanMessage(content=f"Use this answer to make the food recommendations: {messages}")
        ])

        recommended_meals["recommended_meals"] = [json_output]

        return {"recommended_meals": recommended_meals}

In [265]:
# Add nodes and edges
interview_builder = StateGraph(InterviewState)
interview_builder.add_node("ask_question", generate_question)
interview_builder.add_node("search_web", search_web)
interview_builder.add_node("search_wikipedia", search_wikipedia)
interview_builder.add_node("combine_documents", combine_documents)  # New node
interview_builder.add_node("answer_question", generate_answer)
interview_builder.add_node("save_interview", save_interview)
interview_builder.add_node("write_section", write_recommendations)

# Flow
interview_builder.add_edge(START, "ask_question")
interview_builder.add_edge("ask_question", "search_web")
interview_builder.add_edge("search_web", "search_wikipedia")  # Make searches sequential
interview_builder.add_edge("search_wikipedia", "answer_question")
interview_builder.add_conditional_edges("answer_question", route_messages, ['ask_question', 'save_interview'])
interview_builder.add_edge("save_interview", "write_section")
interview_builder.add_edge("write_section", END)

interview_graph = interview_builder.compile().with_config(run_name="Conduct Interviews")

In [268]:
#  View
display(Image(interview_graph.get_graph().draw_mermaid_png()))

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): mermaid.ink:443


ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [258]:
type(meal_type_analysts[0])

__main__.MealTypeAnalysts

In [259]:
type(analysts_messages[0])

dict

In [269]:
analysts_messages[0].keys()

dict_keys(['analyst', 'messages'])

In [270]:
recommended_meals_testing = []
for event in interview_graph.stream({
    "user_profile": user,
    "meal_type_analysts": meal_type_analysts,
    "analysts_messages": analysts_messages,
    "sourced_documents": [],
    "interview": "",
    "recommended_meals": {
        "user_profile": user,
        "recommended_meals": []
    },
    "max_num_turns": 2
}, stream_mode="values"):
    recommended_meals_testing.append(event)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'You are an analyst tasked with interviewing an expert to learn about a specific topic.\n\nYour goal is boil down to interesting and specific insights related to your topic.\n\n1. Interesting: Insights that people will find surprising or non-obvious.\n\n2. Specific: Insights that avoid generalities and include specific examples from the expert.\n\nHere is your topic of focus and set of goals: Explore expert strategies related to high-protein diet — specifically, Protein Pete is here to pump up your breakfast with high-protein options that fit your dairy-free and low-carb lifestyle. He suggests meals like scrambled eggs with spinach and turkey bacon, or a tofu scramble with avocado, ensuring you start your day with the energy you need.\n\nBegin by introducing yourself using a name that fits your persona, and then ask your question.\n\nContinue t

TypeError: list indices must be integers or slices, not str

In [190]:
recommended_meals_testing

[{'user_profile': UserProfile(name='Space Cadet', age=23, gender='Male', height_cm=183, weight_kg=65.0, activity_level='Lightly active', dietary_preferences=['Dairy-Free', 'Low-Carb'], allergies=['Peanuts'], health_conditions=['None'], weight_goal='Maintain weight', past_meals=[]),
  'max_num_turns': 2,
  'sourced_documents': [],
  'meal_type_analysts': [MealTypeAnalysts(meal_type='BREAKFAST', analysts=[Analyst(name='Protein Pete', tone='Energetic', theme='High-Protein Diet', description='Protein Pete is here to pump up your breakfast with high-protein options that fit your dairy-free and low-carb lifestyle. He suggests meals like scrambled eggs with spinach and turkey bacon, or a tofu scramble with avocado, ensuring you start your day with the energy you need.'), Analyst(name='Carb-Conscious Carla', tone='Practical', theme='Low-Carb Options', description='Carb-Conscious Carla focuses on keeping your breakfast low in carbs while still being satisfying and delicious. She recommends dish